In [1]:
import os 
%pwd

'/Users/omkarthakur/Desktop/Text-Summary/research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/omkarthakur/Desktop/Text-Summary'

# Config

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

# Constants:

In [5]:
from src.textsummarization.constants import *
from src.textsummarization.utils.common import read_yaml, create_directories

# Configuraion Manager

In [6]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )
        return model_evaluation_config

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import torch 
import pandas as pd
from src.textsummarization.logging import logger
from tqdm import tqdm
from datasets import load_from_disk,  load_dataset

/Users/omkarthakur/Desktop/Text-Summary/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
! pip install evaluate

In [9]:
import evaluate
import torch

# Define device BEFORE the class
device = "cuda" if torch.cuda.is_available() else "cpu"

# Evaluation
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
            Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                batch_size=16, device=device,
                                column_text="article",
                                column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))
        predictions, true_values = [], []
        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                        attention_mask=inputs["attention_mask"].to(device),
                                        length_penalty=0.8, num_beams=8, max_length=128)
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                for s in summaries]
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            predictions.extend(decoded_summaries)
            true_values.extend(target_batch)

        # finally compute and return rouge scores
        score = metric.compute(predictions=predictions, references=true_values)
        return score
    
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(str(self.config.tokenizer_path))
        model = AutoModelForSeq2SeqLM.from_pretrained(str(self.config.model_path))
        model.to(device)
        logger.info(f"Evaluating on {self.config.data_path}")
        
        # Load the rouge metric
        rouge_metric = evaluate.load('rouge')
        
        # Loading data
        dataset_samsum = load_from_disk(str(self.config.data_path))
     
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        
        # Calculate metrics - use model instead of trainer.model
        score = self.calculate_metric_on_test_ds(
            dataset_samsum["test"][0:10], 
            rouge_metric, 
            model, 
            tokenizer, 
            batch_size=2, 
            device=device,
            column_text="dialogue", 
            column_summary="summary"
        )
        
        # Directly use the scores without accessing fmeasure or mid
        rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index=[f"pegasus"])   
        df.to_csv(self.config.metric_file_name, index=False)     
        
        print("\n" + "="*50)
        print("EVALUATION RESULTS:")
        print("="*50)
        for metric_name, metric_value in rouge_dict.items():
            print(f"{metric_name}: {metric_value:.4f}")
        print("="*50 + "\n")
        

In [10]:
config = ConfigurationManager()
model_evaluation_config = config.get_model_evaluation_config()
model_evaluation = ModelEvaluation(model_evaluation_config)
model_evaluation.evaluate()

[2025-10-14 15:15:09,069: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-10-14 15:15:09,071: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-14 15:15:09,071: INFO: common: created directory at: artifacts]
[2025-10-14 15:15:09,072: INFO: common: created directory at: artifacts/model_evaluation]
[2025-10-14 15:15:09,420: INFO: 4273960055: Evaluating on artifacts/data_transformation/samsum_dataset]


100%|██████████| 5/5 [03:53<00:00, 46.63s/it]

[2025-10-14 15:19:03,063: INFO: rouge_scorer: Using default tokenizer.]

EVALUATION RESULTS:
rouge1: 0.0228
rouge2: 0.0000
rougeL: 0.0228
rougeLsum: 0.0228

